In [1]:
import pandas as pd
import numpy as np

In [39]:
#import file
f = open("../scraped_examples/" + "buckleyChordsLyrics.txt","r")
full_text = f.read()

In [40]:
full_text = full_text.replace("\\r\\n","£")
full_text = "[ch]" + full_text

In [41]:
lyric_sections = full_text.split("££")

In [42]:
tags = ["Intro", "Interlude", "Chorus", "Pre-Chorus", "Verse", "Outro"]

In [43]:
# extract lyric/chord line pairs

chord_list = []
lyric_list = []
section_list = []
i=-1
for section in lyric_sections:
    i+=1
    
    # each section can be 1: just chords, 2: tag then chord/lyric pairs, 
    # 3: no tag and one line of chords, 4: no tag & multiple lines of chords
    # chord/lyric pairs must have line breaks
    
    # check for pairs
    if section.find("[/tab]")!=-1:

        # find potential tag
        space_loc = section.find(" ")
        bracket_loc = section.find("]")
        tag_loc = min(space_loc, bracket_loc)
        tag = section[section.find("[")+1:tag_loc]

        # section has TAG
        if tag in tags:
            # print(str(i) + ": TAG " + tag)

            # remove tag
            new_section = section[section.find("£")+1:]
        
            # extract tab sections
            pair_list = new_section.split("[/tab]")
            for pair in pair_list:
                if len(pair)>0:
                    start_loc = pair.find("[tab]")+5
                    if start_loc == -1:
                        start_loc = 0
                    pound_loc = pair[start_loc:].find("£")
                    chords = pair[start_loc:pound_loc+start_loc]
                    lyrics = pair[pound_loc+start_loc+1:]

                    chords = chords.replace("[ch]","")
                    chords = chords.replace("[/ch]","")
                    chord_list += [chords]
                    lyric_list += [lyrics]

        # section has NO TAG
        else:
            # print(str(i) + ": no tag, multiple lines " + tag)
            if section.find("[tab]")!=-1: print("contains pairs")
            
    # section has no pairs 
    # else: print(str(i) + ": no pairs")

In [44]:
# pair words & chords together

pair_tuples = ()
# iterate through lyric/chord list
for line in range(len(lyric_list)):
    # split lyrics and chords on each space character
    phrase = lyric_list[line]
    # find lyric space locations
    space_locs = [0] + [i+1 for i in range(len(phrase)) if (phrase[i]==" " and phrase[i+1]!=" ")] + [len(phrase)]
    
    split_words = [phrase[space_locs[i]:space_locs[i+1]] for i in range(len(space_locs)-1)]
    
    chord_phrase = chord_list[line]
    split_chords = []
    for i in range(len(space_locs)-1):
        #chord = chord_phrase[space_locs[i]:space_locs[i+1]]
        # if interrupting a chord, find next space
        if len(chord_phrase)>space_locs[i+1]:
            if chord_phrase[space_locs[i+1]-1]!=" " and chord_phrase[space_locs[i+1]]!=" ":
                new_space_loc = chord_phrase[space_locs[i+1]:].find(" ")
                if new_space_loc==-1:
                    new_space_loc = len(chord_phrase)-space_locs[i+1]
                # chord = chord_phrase[space_locs[i]:space_locs[i+1]+new_space_loc]
                space_locs[i+1] += new_space_loc
        split_chords += [chord_phrase[space_locs[i]:space_locs[i+1]]]
    
    pair_tuples += tuple(zip(split_words, split_chords))

In [45]:
# trim spaces, lower, and remove commas

cleaned_pairs = [[str.strip(pair[i]).lower().replace(",","") 
                  if i==0 else str.strip(pair[i]) 
                  for i in range(len(pair))] for pair in pair_tuples]

In [46]:
# add repeats of pairs when one word has multiple chords
chord_pairs = []
for pair in cleaned_pairs:
    # find multiple chords with one word
    split_chords = pair[1].split()
    if pair[1]!='' and split_chords!=[pair[1]]:
        for chord in split_chords:
            chord_pairs.append([pair[0], chord])
    else:
        chord_pairs.append(pair)

In [47]:
# find first chord from intro

# is first chord empty
if chord_pairs[0][1]=='':
    # find first [tab] instance
    tab_loc = full_text.find("[tab]")
    # then find previous [/ch] instance (flip text then find first instance)
    ch_search_text = full_text[:tab_loc][::-1]
    ch_loc = ch_search_text.find("]hc/[")
    # extract chord just after that
    chord_backwards = ch_search_text[ch_loc+5:ch_loc+5+ch_search_text[ch_loc+5:].find("]")]
    # insert in first chord space
    chord_pairs[0][1] = chord_backwards[::-1]

In [48]:
chord_pairs

[['i', 'Am'],
 ['heard', 'C'],
 ['there', ''],
 ['was', ''],
 ['a', ''],
 ['secret', 'Am'],
 ['chord', ''],
 ['that', ''],
 ['david', 'C'],
 ['played', ''],
 ['and', ''],
 ['it', ''],
 ['pleased', 'Am'],
 ['the', ''],
 ['lord', ''],
 ['but', ''],
 ['you', 'F'],
 ["don't", ''],
 ['really', ''],
 ['care', 'G'],
 ['for', ''],
 ['music', ''],
 ['do', 'C'],
 ['you?', ''],
 ['well', ''],
 ['it', ''],
 ['goes', 'C'],
 ['like', ''],
 ['this', ''],
 ['the', ''],
 ['fourth', 'F'],
 ['the', ''],
 ['fifth', 'G'],
 ['the', ''],
 ['minor', 'Am'],
 ['fall', ''],
 ['and', ''],
 ['the', ''],
 ['major', 'F'],
 ['lift', ''],
 ['the', ''],
 ['baffled', 'G'],
 ['king', ''],
 ['composing', 'E7'],
 ['hallelujah', 'Am'],
 ['hallelujah', 'F'],
 ['hallelujah', 'Am'],
 ['hallelujah', 'F'],
 ['hallelu-u-u-u-jah', 'C'],
 ['hallelu-u-u-u-jah', 'G'],
 ['hallelu-u-u-u-jah', 'C'],
 ['....', 'Am'],
 ['well', ''],
 ['your', ''],
 ['faith', 'C'],
 ['was', ''],
 ['strong', ''],
 ['but', ''],
 ['you', ''],
 ['needed', 'Am'

In [49]:
# CLEANING
# trim spaces
# convert to all lowercase
# remove commas and dashes?: unnecessary characters
# forward filling

In [50]:
pair_df = pd.DataFrame(chord_pairs, columns=['lyric','chord'])

In [51]:
pair_df.chord = pair_df['chord'].replace('',np.nan).ffill()

In [52]:
pair_df

,lyric,chord
0,i,Am
1,heard,C
2,there,C
3,was,C
4,a,C
...,...,...
275,....,C
276,,Am
277,halleluuuuuuujah,F
278,halleluuuuuuujah,G
